In [44]:
import pandas as pd
import hvplot.pandas
from pathlib import Path
%matplotlib inline
%store -r dates_FOMC
from datetime import timedelta

In [45]:
dates_FOMC = dates_FOMC.sort_values()
dates_FOMC[38]

Timestamp('2021-01-27 00:00:00')

In [46]:
path = Path('VIX_History.csv')
vix_df = pd.read_csv(path, index_col = 'DATE', parse_dates = True, infer_datetime_format = True)
vix_df.head()

,OPEN,HIGH,LOW,CLOSE
DATE,,,,
1990-01-02,17.24,17.24,17.24,17.24
1990-01-03,18.19,18.19,18.19,18.19
1990-01-04,19.22,19.22,19.22,19.22
1990-01-05,20.11,20.11,20.11,20.11
1990-01-08,20.26,20.26,20.26,20.26


In [47]:
vix_smaller_df = pd.DataFrame(vix_df.loc['2017-01-01':'2022-05-05'].iloc[:,3])

dates_FOMC_df = pd.DataFrame(dates_FOMC)

vix_smaller_df.head()
# for date in dates_FOMC:
#     dates_FOMC.index(
    

,CLOSE
DATE,
2017-01-03,12.85
2017-01-04,11.85
2017-01-05,11.67
2017-01-06,11.32
2017-01-09,11.56


In [48]:
def date_bump_down(date):
    if date.dayofweek == 0:
        date_down = date - timedelta(days = 3)
    else:
        date_down = date - timedelta(days = 1)
    return date_down

def date_bump_up(date):
    if date.dayofweek == 4:
        date_up = date + timedelta(days = 3)
    else:
        date_up = date + timedelta(days = 1)
    return date_up

In [49]:
b4 = []
dayof = []
after = []

for date in dates_FOMC:
    b4.append(vix_smaller_df.loc[date_bump_down(date),'CLOSE'])
    dayof.append(vix_smaller_df.loc[date,'CLOSE'])
    after.append(vix_smaller_df.loc[date_bump_up(date),'CLOSE'])
    
print(len(b4)) 
print(len(dayof))
print(len(after))
d = {'before': b4, 'day of': dayof, 'after': after}

trend = pd.DataFrame(d)
trendy = trend.describe(include='all')
trend_mean = trendy.iloc[1,:]
trend_5 = trendy.iloc[1,:] - 1 * trendy.iloc[2,:]
trend_95 = trendy.iloc[1,:] + 1 * trendy.iloc[2,:]
trendy
#trend_mean.hvplot()
# find a way to normalize the data here
trend_mean.hvplot()*trend_5.hvplot()*trend_95.hvplot()

49
49
49


:Overlay
   .Curve.Mean   :Curve   [index]   (mean)
   .Curve.A_0.I  :Curve   [index]   (0)
   .Curve.A_0.II :Curve   [index]   (0)

In [65]:
vix_change_wide = []
for date in dates_FOMC:
    change = (vix_smaller_df.loc[date_bump_up(date),'CLOSE'] - vix_smaller_df.loc[date_bump_down(date),'CLOSE'])/vix_smaller_df.loc[date_bump_down(date),'CLOSE']
    vix_change_wide.append(change)

vix_change_wide[:4]
# vix_change_wide_df = pd.DataFrame(vix_change_wide, )
# vix_change_wide_df

[-0.005004170141784862,
 -0.08861788617886177,
 -0.012275731822473938,
 0.04606525911708258]

In [51]:
vix_df.iloc[:,0]

DATE
1990-01-02    17.24
1990-01-03    18.19
1990-01-04    19.22
1990-01-05    20.11
1990-01-08    20.26
              ...  
2022-05-03    31.76
2022-05-04    29.12
2022-05-05    25.97
2022-05-06    32.23
2022-05-09    31.90
Name: OPEN, Length: 8148, dtype: float64

In [64]:
vix_change = []
for date in dates_FOMC:
    change = (vix_df.iloc[:,3].loc[date] - vix_df.iloc[:,0].loc[date])/vix_df.iloc[:,3].loc[date]
    vix_change.append(change)
vix_change[:4]

[0.0016934801016089203,
 -0.04213241616509015,
 0.011235955056179702,
 0.029135338345864705]

In [53]:
dates_FOMC.weekday

Int64Index([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2,
            4, 2, 2, 2, 1, 2, 3, 0, 1, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2,
            2, 2, 2, 2, 2],
           dtype='int64')

In [54]:
vix_change_df = pd.DataFrame(vix_change)
vix_change_wide_df = pd.DataFrame(vix_change_wide)
display(vix_change_df.head())
vix_change_df.describe()

,0
0,0.001693
1,-0.042132
2,0.011236
3,0.029135
4,0.036458


,0
count,49.000000
mean,-0.003449
std,0.088812
min,-0.202793
25%,-0.050960
50%,0.000725
75%,0.039293
max,0.359850


In [63]:
sum = (vix_change_df.iloc[:,0] + 1).cumprod()
sum_df = pd.DataFrame(sum)
sum_df.rename(columns = {0:'Cumulative return'}, inplace = True)
sum_df['Date'] = dates_FOMC
sum_df.set_index('Date', inplace = True)
sum_df

sum_wide = (vix_change_wide_df.iloc[:,0] + 1).cumprod()
sum_wide_df = pd.DataFrame(sum_wide)
sum_wide_df.rename(columns = {0:'Cumulative return'}, inplace = True)
sum_wide_df['Date'] = dates_FOMC
sum_wide_df.set_index('Date', inplace = True)
sum_wide_df.head()

,Cumulative return
Date,
2017-02-01,0.994996
2017-03-15,0.906821
2017-05-03,0.895690
2017-06-14,0.936950
2017-07-26,1.004513


In [56]:
test2 = sum_df.sample(n=sum_df.shape[0], replace=True)
test2.describe()

,Cumulative return
count,49.000000
mean,1.076943
std,0.142464
min,0.809694
25%,1.001693
50%,1.088889
75%,1.180830
max,1.433129


In [57]:
sum_df.hvplot()*sum_wide_df.hvplot()#add legend

:Overlay
   .Curve.I  :Curve   [Date]   (Cumulative return)
   .Curve.II :Curve   [Date]   (Cumulative return)

In [58]:
sum_wide_df.hvplot(color="red")

:Curve   [Date]   (Cumulative return)

In [59]:
vix_change_df.hvplot.hist()



:Histogram   [0]   (0_count)

In [60]:
vix_change_wide_df.hvplot.hist()

:Histogram   [0]   (0_count)

In [61]:


vix_change_wide_df.hvplot.hist()*vix_change_df.hvplot.hist()


#find a way to measure how much skewness there is

:Overlay
   .Histogram.I  :Histogram   [0]   (0_count)
   .Histogram.II :Histogram   [0]   (0_count)

In [62]:
vix_wide_skew = vix_change_wide_df.skew()
print("Vix Change Wide Skew Value")
display(vix_wide_skew)

vix_change_skew =vix_change_df.skew()
print("Vix Change Skew Value")
display(vix_change_skew)

Vix Change Wide Skew Value


0    1.699213
dtype: float64

Vix Change Skew Value


0    1.165366
dtype: float64